### Song based sentiment analysis using Textblob

In [31]:
# Imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

import nltk
from nltk.corpus import stopwords
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora

from textblob import TextBlob
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

In [2]:
#Samaras Laptop
#song_data = pd.read_csv('C:\\Users\sdo\Studienarbeit\DIE_DATEN\datensatz.csv')
#song_data = song_data[(song_data['Language'] == 'en')]
#song_data.to_csv("outData.csv", index=False)
#song_data = pd.read_csv('C:\\Users\sdo\Studienarbeit\outData.csv')


#Anjas Laptop
# song_data = pd.read_csv('../Daten/train.csv')
# song_data = song_data[(song_data['Language'] == 'en')]
# song_data.to_csv("../Daten/outData.csv", index=False)
song_data = pd.read_csv('../Daten/cleanedData.csv')

## Get a balanced sample of n songs per genre

In [15]:
num_genres = song_data.Genre.nunique()
num_topics = num_genres

In [16]:
song_data = song_data[["Genre", "Lyrics"]]
song_data = song_data.groupby('Genre').sample(5).reset_index(drop=True)
song_data.head(10)

,Genre,Lyrics
0,Country,We crossed the Powder River in the fall of '81...
1,Country,"It's too late, you say you want me, but it's t..."
2,Country,Well a hundred years from now I won't be cryin...
3,Country,"(C.W. McCall, Bill Fries, Chip Davis) It's 4 A..."
4,Country,Her telephone rang 'bout a quarter to nine She...
5,Electronic,"Late nights, everyday is a motherfuckin' party..."
6,Electronic,"Born under punchs, cross eyed and painless Sli..."
7,Electronic,I take a drink I need to wake up this machine ...
8,Electronic,Tell me how much more do I have to lose Before...
9,Electronic,[Chorus] Woman you getting paid not played Pus...


## Remove punctuation/lower casing

In [17]:
song_data_processed = song_data.copy()
song_data_processed['Lyrics'] = song_data_processed['Lyrics'].map(lambda x: re.sub('[,\.!?]', '', x))
song_data_processed['Lyrics'] = song_data_processed['Lyrics'].map(lambda x: x.lower())

## Remove stopwords and Tokenize

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anjan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
def calc_sentiment(row):
    # Split the text into lines
    lines = row['Lyrics'].split('\n')
    # Calculate sentiment polarity for each line
    sentiments = [TextBlob(line).sentiment.polarity for line in lines]
    return sentiments

In [22]:
song_data_processed['Sentiment'] = song_data_processed.apply(calc_sentiment, axis=1)

In [23]:
song_data_processed

,Genre,Lyrics,Sentiment
0,Country,we crossed the powder river in the fall of '81...,[0.022030651340996167]
1,Country,it's too late you say you want me but it's too...,[0.03633333333333335]
2,Country,well a hundred years from now i won't be cryin...,[0.026623376623376622]
3,Country,(cw mccall bill fries chip davis) it's 4 am an...,[-0.049699265381083574]
4,Country,her telephone rang 'bout a quarter to nine she...,[-0.15491553465691393]
5,Electronic,late nights everyday is a motherfuckin' party ...,[0.1381098484848485]
6,Electronic,born under punchs cross eyed and painless slip...,[0.12962962962962965]
7,Electronic,i take a drink i need to wake up this machine ...,[-0.016666666666666666]
8,Electronic,tell me how much more do i have to lose before...,[0.3187863860277654]
9,Electronic,[chorus] woman you getting paid not played pus...,[-0.03276748971193417]


### Explore the results

In [ ]:
##This is hard to achieve as the sentiment vectors have different length due to different lengths of the songs

## Trying classification

In [43]:
X = song_data_processed[["Sentiment"]]
#X = np.array(X)
label_encoder = LabelEncoder()
song_data_processed["GenreID"] = label_encoder.fit_transform(song_data_processed["Genre"])
y = song_data_processed["GenreID"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [44]:
X

,Sentiment
0,[0.022030651340996167]
1,[0.03633333333333335]
2,[0.026623376623376622]
3,[-0.049699265381083574]
4,[-0.15491553465691393]
5,[0.1381098484848485]
6,[0.12962962962962965]
7,[-0.016666666666666666]
8,[0.3187863860277654]
9,[-0.03276748971193417]


In [40]:
#K-nearest neighbors
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train, y_train)
neigh.score(X_test, y_test)

Unexpected exception formatting exception. Falling back to standard exception


TypeError: float() argument must be a string or a real number, not 'list'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\anjan\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\anjan\AppData\Local\Temp\ipykernel_12764\3720955400.py", line 3, in <module>
    neigh.fit(X_train, y_train)
  File "c:\Users\anjan\anaconda3\envs\Studienarbeit\Lib\site-packages\sklearn\neighbors\_classification.py", line 215, in fit
    return self._fit(X, y)
           ^^^^^^^^^^^^^^^
  File "c:\Users\anjan\anaconda3\envs\Studienarbeit\Lib\site-packages\sklearn\neighbors\_base.py", line 454, in _fit
    X, y = self._validate_data(
           ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\anjan\anaconda3\envs\Studienarbeit\Lib\site-packages\sklearn\base.py", line 584, in _validate_data
    X, y = check_X_y(X, y, **check_

In [45]:
#svm classifier
svc = SVC(kernel="linear")
svc.fit(X_train, y_train)
svc.score(X_test, y_test)

0.14466666666666667

In [41]:
#Random forest
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.13066666666666665

### Was hier gemacht wurde:
- Nehmen eines song subsets mit jeweils n songs aus jedem der m genres
- Bereinigen der Lyrics mit verschiedenen Preprocessing methoden (Best Practice)
- Anwendung der sentiment analyse mithilfe der python textblob bibliothekt (Ergebnis in form von polarity & subjectivity)
- Dabei: Anwendung der Analyse auf jede Zeile (getrennt durch \n) anstatt des gesamten Liedes

### Ergebnis
- Datenqualität Problem: Manche Songs verwenden keine \n zum indizieren einer neuen Zeile sondern bloß \
- Die Output-sentiment-vektoren sind unterschiedlich lang, was eine analyse und den vergleich nicht wirklich möglich macht
- Die Methode wird deshalb zugunsten der gesamt-song-sentiments fallen gelassen